# Qiskit to LLVM IR

We want to examine how we can bring a Qiskit circuit down into LLVM IR. We will do this via qcor, and follow the following pipeline:
```
Qiskit Circuit --> OpenQASM3 --> MLIR --> QIR / LLVM
```

First, we define and transpile our circuit

In [1]:
from qiskit import QuantumCircuit
from qiskit.test.mock import FakeBoeblingen
from qiskit.compiler import transpile


s = '011'   # the hidden binary string
n = 3 # number of bits used to represent s


# We need a circuit with n qubits, plus one auxiliary qubit
# We also need n classical bits to receive the output
qc = QuantumCircuit(n+1, n)

# Put auxiliary in the minus state |->
# We leave the other n qubits in just |0>
qc.h(n)
qc.z(n)

# Apply Hadamard gates to each of the n "main" qubits (excluding the auxiliary)
# before querying the oracle
for i in range(n):
    qc.h(i)
    
# Apply barrier
qc.barrier()

# Apply the inner-product oracle
s = s[::-1] # reverse s to fit qiskit's qubit ordering
for q in range(n):
    if s[q] == '0':
        qc.i(q)
    else:
        qc.cx(q, n)
        
# Apply barrier
qc.barrier()

# Apply Hadamard gates after querying the oracle
for i in range(n):
    qc.h(i)


# Measurement
for i in range(n):
    qc.measure(i, i)


qc.draw()

simulated_backend = FakeBoeblingen()
transpiled_qc3 = transpile(qc, simulated_backend, optimization_level=3)

Now that we have a transpiler circuit, we can go ahead and dump it into OpenQASM. From there, the process will look very similar to what we saw in the [OpenQASM example](../../OpenQASM/).

In [2]:
# Qiskit Circuit  <-->  .qasm file
from qiskit import qasm3

# Qiskit -> OpenQASM3
# Print
print(qasm3.dumps(transpiled_qc3))
# Dump to file
with open("Bernstein_Vazirani.qasm", 'w+') as fp:
   qasm3.dump(transpiled_qc3, fp)

# Qiskit <- OpenQASM3 support?

# ----------------------------------------------------------------------------

# Deprecated

# Qiskit -> OpenQASM2
# transpiled_qc3.qasm(formatted=True, filename='Bernstein_Vazirani.qasm')
# Qiskit <- OpenQASM2
# new_qc = QuantumCircuit.from_qasm_file('Bernstein_Vazirani.qasm')

OPENQASM 3;
include "stdgates.inc";
bit[3] c;
u2(0, pi) $0;
u2(-pi, -pi) $1;
u2(0, pi) $6;
u2(0, pi) $15;
barrier $6, $0, $15, $1;
cx $6, $1;
cx $0, $1;
barrier $6, $0, $15, $1;
u2(0, pi) $0;
u2(0, pi) $15;
u2(0, pi) $6;
c[0] = measure $6;
c[1] = measure $0;
c[2] = measure $15;



### Caveat

Due to several nuances with qcor, some changes need to be made to the above raw OpenQASM dump before it can be converted to LLVM. Namely,

+ OpenQASM2 syntax must be used (i.e. creg c[3] instead of bit[3] c)
+ However, the version statement must be kept at OPENQASM3.0
+ All references to physical qubits ($ and then a number) must be replaced with virtual qubits (define qreg q[20] at the top and replace $x with q[x])
+ qcor has no implicit casting so replace any integers with floats (i.e. replace each 0 with 0.0)

# OpenQASM3 to LLVM IR

Here is where qcor comes in. We will not give an in depth tutorial on setting up qcor here: for resources, see [their documentation](https://qcor.readthedocs.io/en/latest/install.html). We recommend using the Docker image.

Once this is set up, we will use the `qcor-mlir-tool` to gain insight into the intermediate stages of compilation.

### MLIR and LLVM

[MLIR](https://mlir.llvm.org/) is a much broader LLVM project, and mostly out of our scope. It has a multitude of uses. For our purposes, we are interested in how it can be leveraged to compile quantum code. qcor leverages MLIR as a way to lower OpenQASM code to LLVM (technically, [QIR](https://devblogs.microsoft.com/qsharp/introducing-quantum-intermediate-representation-qir/), which adds several necessary features to LLVM for quantum programming). For information about that, see [this presentation](https://mlir.llvm.org/OpenMeetings/2021-05-27-Quantum-Classical-Compilation-with-MLIR.pdf).

We will use the `qcor-mlir-tool`. It has a flag `-emit` with three settings:
``` terminal
--emit=<value>                                       - Select the kind of output desired
    =mlir                                              -   output the MLIR dump
    =llvm                                              -   output the LLVM IR dump
    =mlir-llvm                                         -   output the MLIR LLVM Dialect dump
```
After running on our `Bernstein_Vazirani.qasm` file, we receive the following outputs: 
+ [MLIR](Bernstein_Vazirani.mlir)
+ [LLVM](Bernstein_Vazirani.ll)
+ [MLIR-LLVM](Bernstein_Vazirani-ll.mlir)